In [114]:
import numpy as np
import pandas as pd
import geopandas
import statsmodels.formula.api as smf

In [115]:
def kernel(R, c, h):
    indicator = (np.abs(R-c) <= h).astype(float)
    return indicator * (1 - np.abs(R-c)/h)

In [116]:
def RD_approach(df,bw_value=28):   
    
    rdd_df = df.assign(threshold=(df["date"] > 0).astype(int))
    model = smf.wls("r~date*threshold", rdd_df,weights=kernel(df["date"], c=0, h=bw_value)).fit()

    return pd.DataFrame(model.params).rename(columns={0:'coeff'}).loc[['threshold','Intercept'],:].join(model.conf_int(alpha=0.05).rename(columns={0:'[0.05', 1:'0.95]'}))

In [117]:
idx=pd.IndexSlice

# Reading data

In [118]:
US_maps=geopandas.read_file('./wildfires_project/Data/USA_shapefile/county_level/')

In [119]:
social_distancing_df=pd.read_csv('./wildfires_project/Data/social_distancing_county_2019_2020_2021.csv')
social_distancing_df.date=pd.to_datetime(social_distancing_df.date)
social_distancing_df.set_index(['county_fips','date'],inplace=True)
social_distancing_df.sort_index(inplace=True)

In [120]:
hit_county_WA=pd.read_csv('./wildfires_project/Data/affected_counties_WA.csv')

In [121]:
hit_county_OR=pd.read_csv('./wildfires_project/Data/affected_counties_OR.csv')

In [122]:
indoor_activity=pd.read_csv('./wildfires_project/Data/indoor_activity_index/indoor_activity_2018_2021_not_smoothed.csv')
indoor_activity_1=pd.read_csv('./wildfires_project/Data/indoor_activity_index/indoor_activity_2018_not_smoothed.csv').rename(columns={'countyFIPS':'county','inprop_outprop_centered':'r'})
indoor_activity_2=pd.read_csv('./wildfires_project/Data/indoor_activity_index/indoor_activity_2019_not_smoothed.csv').rename(columns={'countyFIPS':'county','inprop_outprop_centered':'r'})
indoor_activity=pd.concat([indoor_activity,indoor_activity_1,indoor_activity_2])
indoor_activity.date=pd.to_datetime(indoor_activity.date)
indoor_activity.set_index(['county','date'], inplace=True)
indoor_activity.sort_index(inplace=True)
indoor_activity['week']=np.array(indoor_activity.index.get_level_values(1).isocalendar().week)

# Regression Discontinuity for Washington

In [123]:
results_IA_WA_counties=[]
for i in range(len(hit_county_WA.county)):    
    RD_setup_IA_df=indoor_activity[(indoor_activity.index.get_level_values(1)>='2020-06-01')&(indoor_activity.index.get_level_values(1)<='2020-12-30')].loc[idx[hit_county_WA.county[i],:],['r']].droplevel(0)
    RD_setup_IA_df.index=(RD_setup_IA_df.index-pd.to_datetime('2020-09-10')).total_seconds()/86400
    RD_setup_IA_df.reset_index(inplace=True)
    results_IA_WA_counties.append(RD_approach(RD_setup_IA_df,28))

In [124]:
results_IA_WA_counties=pd.concat(results_IA_WA_counties)
results_IA_WA_counties.index=[np.repeat(hit_county_WA.county,2),results_IA_WA_counties.index]

In [125]:
results_IA_WA_counties

coeff     [0.05     0.95]
county                                        
53033  threshold  0.149174  0.100468  0.197879
       Intercept  0.984758  0.952210  1.017306
53063  threshold  0.130216  0.076052  0.184380
       Intercept  0.930846  0.894650  0.967041
53011  threshold  0.097515  0.016246  0.178784
       Intercept  0.954609  0.900300  1.008918
53067  threshold  0.150016  0.091560  0.208473
       Intercept  0.849217  0.810153  0.888281
53077  threshold  0.021650  0.004092  0.039207
       Intercept  0.985441  0.973708  0.997174

In [126]:
#(results_IA_WA_counties.groupby('county').agg({'coeff':'first'})/results_IA_WA_counties.groupby('county').agg({'coeff':'last'})).to_csv('./wildfires_project/Data/Relative_Varitation_indoor_activities_WA.csv')

In [127]:
results_IA_WA_counties.to_csv('./wildfires_project/Data/results_RD_WA_indoor_activities.csv')

# Regression Discontinuity for Oregon

In [128]:
results_IA_OR_counties=[]
models=[]
for i in range(len(hit_county_OR.county)):    
    RD_setup_IA_df=indoor_activity[(indoor_activity.index.get_level_values(1)>='2020-06-01')&(indoor_activity.index.get_level_values(1)<='2020-12-30')].loc[idx[hit_county_OR.county[i],:],['r']].droplevel(0)
    RD_setup_IA_df.index=(RD_setup_IA_df.index-pd.to_datetime('2020-09-10')).total_seconds()/86400
    RD_setup_IA_df.reset_index(inplace=True)
    model_results=RD_approach(RD_setup_IA_df,28)
    results_IA_OR_counties.append(model_results)

In [129]:
results_IA_OR_counties=pd.concat(results_IA_OR_counties)
results_IA_OR_counties.index=[np.repeat(hit_county_OR.county,2),results_IA_OR_counties.index]

In [130]:
results_IA_OR_counties

coeff     [0.05     0.95]
county                                        
41051  threshold  0.094503  0.001251  0.187754
       Intercept  0.989218  0.926901  1.051534
41067  threshold  0.070382 -0.005122  0.145887
       Intercept  1.065776  1.015319  1.116233
41005  threshold  0.297404  0.230690  0.364119
       Intercept  0.915694  0.871111  0.960277
41039  threshold  0.066387 -0.006087  0.138861
       Intercept  1.053138  1.004707  1.101570
41047  threshold  0.019234 -0.064098  0.102565
       Intercept  1.064749  1.009062  1.120436

In [131]:
results_IA_OR_counties.to_csv('./wildfires_project/Data/results_RD_OR_indoor_activities.csv')

In [132]:
#(results_IA_OR_counties.groupby('county').agg({'coeff':'first'})/results_IA_OR_counties.groupby('county').agg({'coeff':'last'})).to_csv('./wildfires_project/Data/Relative_Variation_indoor_activities_OR.csv')